Imports

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import date

In [2]:
#loading_branch_service
df_branch_service = pd.read_json("branch_service_transaction_info.json")

#loading_customer_transaction
df_customer_transaction = pd.read_json("customer_transaction_info.json")

In [3]:
#drop_dups_branch_service
df_branch_service = df_branch_service.drop_duplicates(subset=['txn_id'])

#drop_dups_customer_transaction
df_customer_transaction = df_customer_transaction.drop_duplicates(subset=['txn_id'])

In [4]:
#merge_dataframe
df_merged = pd.merge(df_customer_transaction, df_branch_service)

In [5]:
#fill_branch_name
df_merged['branch_name'] = df_merged.replace('',np.nan).groupby('txn_id')['branch_name'].transform('first')
df_merged['branch_name'] = df_merged['branch_name'].ffill().bfill()

In [6]:
#fill_price
df_merged['price'] = df_merged['price'].fillna(df_merged.groupby(['branch_name','service'])['price'].transform('mean'))

In [7]:
#standardize_last_name
df_merged['last_name'] = df_merged['last_name'].str.replace('\W', '', regex=True)
df_merged['last_name'] = df_merged['last_name'].str.upper()

In [8]:
#standardize_first_name
df_merged['first_name'] = df_merged['first_name'].str.replace('\W', '', regex=True)
df_merged['first_name'] = df_merged['first_name'].str.upper()

In [9]:
#validate_dates
today = str(date.today())
df_merged['avail_date'] = pd.to_datetime(df_merged['avail_date'], format='%Y-%m-%d')
df_merged['birthday'] = pd.to_datetime(df_merged['birthday'], format='%Y-%m-%d')
df_merged = df_merged[(df_merged['avail_date'] <= today) & (df_merged['birthday'] <= today)]
df_merged = df_merged[(df_merged['avail_date'] > df_merged['birthday'])]

In [10]:
#validate_price
df_merged['price'] = df_merged['price'].round(2)

In [11]:
#ingestion_to_db
db_connect = sqlite3.connect('labex3')
cur = db_connect.cursor()
cur.execute("create table if not exists labex3(txn_id varchar(45) not null primary key, avail_date datetime default current_date, last_name varchar(20), first_name varchar(20), birthday datetime default current_date, branch_name varchar(30), service varchar(30), price double)")
db_connect.commit()
df_merged.to_sql('labex3', db_connect, if_exists='replace', index=False)

43136